In [1]:
api_key = "OPENAI_API_KEY"

In [2]:
!apt-get install -y poppler-utils
!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
from pdf2image import convert_from_path
import os
import base64
import requests
from IPython.display import Markdown, display

In [4]:
def convert_pdf_to_images(pdf_file, output_folder):
    pages = convert_from_path(pdf_file)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, page in enumerate(pages):
        page.save(os.path.join(output_folder, f'page_{i}.jpg'), 'JPEG')

    print(f"Images saved in folder: {output_folder}")

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [5]:
import os
import requests

def automate_process(pdf_file_path, api_key, output_folder):
    output_folder = output_folder

    convert_pdf_to_images(pdf_file_path, output_folder)

    image_paths = [os.path.join(output_folder, f'page_{i}.jpg') for i in range(len(os.listdir(output_folder)))]
    image_base64_list = [encode_image(image_path) for image_path in image_paths]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    for img in image_base64_list:
        payload = {
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": """Please provide me summary of this candidate.
                            I want to get with this order.
                            1. Education
                            2. Work Experiences
                            3. Other relevant information that employer and HR need to know."""
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{img}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 1000
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        return response.json()

In [6]:
pdf_file_path = '/content/Resume.pdf'
output_folder = '/content/output_images'
output = automate_process(pdf_file_path, api_key, output_folder)

Images saved in folder: /content/output_images


In [7]:
content = output['choices'][0]['message']['content']

display(Markdown(content))

**Summary of Candidate:**

1. **Education:**
   - **The Pre-Collegiate Program of Yangon (PCP):** 
     - Duration: March 2023 – March 2024
     - CGPA: 3.59
     - Description: A 12-month intensive liberal arts program aimed at creating agents of change in Myanmar.
     - Activities: Participated in discussion-based courses including gender studies, history, civic engagement, academic writing, environmental studies, psychology, and religious studies.
   
   - **Basic Education High School, Ahlone:**
     - Duration: June 2008 - March 2018
     - Matriculation Score: 402
     - Roles: Chosen as one of the team leaders in the classroom every school year
     - Activities: Participated in school activities such as gardening, cleaning, and organizing school events.

2. **Work Experiences:**
   - **Inya Economics, Volunteer:**
     - Location: Yangon, Myanmar
     - Duration: June 2023 - Present
     - Responsibilities: Worked as a program assistant in Sustainable & Responsible Business Training, participated in a research proposal, calculated budget, and collected data for a poverty research report based on energy prices.

   - **Myanmar Youth Initiative, Program Officer:**
     - Location: Yangon, Myanmar
     - Duration: January 2023 - Present
     - Responsibilities: Coordinated and evaluated participants of the Financial Literacy and Business Leadership Training for women, served as a program coordinator for Career Readiness Program (Phase-2), and facilitated productive meetings between mentors and program participants.

   - **Hana Microfinance, Data Analyst Intern:**
     - Location: Yangon, Myanmar
     - Duration: August 2022 - February 2023
     - Responsibilities: Prepared courses for data literacy training, conducted research to streamline team processes, implemented a Python program for feasibility studies, performed proof-of-concept assessments, and developed dynamic dashboards from company’s internal website.

3. **Other Relevant Information:**
   - Demonstrated leadership skills as one of the team leaders in the classroom every school year during high school.
   - Actively engaged in various roles including volunteer work, program coordination, and data analysis.
   - Possesses strong skills in data analysis, research, program coordination, and training.
   - Experienced with innovative methodologies to improve efficiency and streamline processes.